In [1]:
import pandas as pd
import numpy as np


In [2]:
# adjust the path to where you saved the dataset
df = pd.read_excel(r"C:\Users\Hp\OneDrive\Desktop\projects\UCI Online Retail Dataset\Online Retail.xlsx")


In [3]:
# check the first few rows
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [4]:
# shape of the dataset (rows, columns)
print("Shape:", df.shape)

Shape: (541909, 8)


541,909 rows and 8 columns.

This is a medium-to-large dataset, great for demonstrating ETL. It represents half a million retail transactions.

In [5]:
# check column names and data types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


In [6]:
# check missing values
print(df.isnull().sum())


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


Description → 1,454 missing (~0.3%). Minor, can drop or fill with placeholder.

CustomerID → 135,080 missing (~25%). This is significant. These are anonymous transactions (common in retail). We may decide to:

Keep them (if focusing on sales trends).

Drop them (if focusing on customer segmentation).

In [7]:
# quick statistical summary
print(df.describe())

            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16791.000000
max     80995.000000   38970.000000   18287.000000


Quantity:

Min = -80,995 → clearly invalid. These are returns or data errors.

Max = 80,995 → possible bulk order, but needs checking.

UnitPrice:

Min = -11,062.06 → invalid (no product can have negative price).

Max = 38,970 → extremely high (maybe data entry error or special case).

CustomerID: Range is 12,346 – 18,287. Looks consistent, but missing values are many

# Step 2: Data Cleaning & Transformation

In [8]:
# 1. Remove canceled transactions (InvoiceNo starting with "C")
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [9]:
# 2. Remove rows where Quantity <= 0
df = df[df['Quantity'] > 0]

In [10]:
# 3. Remove rows where UnitPrice <= 0
df = df[df['UnitPrice'] > 0]

In [11]:
# 4. Fill missing values in Description with "Unknown Product"
df['Description'] = df['Description'].fillna('Unknown Product')

In [12]:
# 5. Handle missing CustomerID (Option A: mark as 'Guest Customer' for sales analysis)
df['CustomerID'] = df['CustomerID'].fillna(0)  # 0 means 'Guest Customer'

In [13]:
# 6. Convert CustomerID to integer
df['CustomerID'] = df['CustomerID'].astype(int)

In [14]:
# 7. Drop duplicates
df = df.drop_duplicates()

Canceled transactions and invalid sales are removed to keep only business-relevant records.

Product descriptions are standardized, and unknown customers are tracked separately for analysis.

Duplicates are dropped to ensure each record is unique and valid

## Final check and summary

In [15]:
print("Shape after cleaning:", df.shape)

Shape after cleaning: (524878, 8)


In [16]:
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [17]:
print(df.describe())

            Quantity      UnitPrice     CustomerID
count  524878.000000  524878.000000  524878.000000
mean       10.616600       3.922573   11437.732164
std       156.280031      36.093028    6799.513627
min         1.000000       0.001000       0.000000
25%         1.000000       1.250000       0.000000
50%         4.000000       2.080000   14350.000000
75%        11.000000       4.130000   16245.000000
max     80995.000000   13541.330000   18287.000000


After applying the defined cleaning rules, the dataset shape reduced from the initial ~541,909 rows to 524,878 rows, reflecting the removal of roughly 17,000+ invalid or irrelevant transactions. This reduction demonstrates effective filtering of noisy and non-business relevant data while retaining the vast majority of meaningful sales records.

Missing Values :-

All missing values have been successfully addressed. Each column now has zero nulls, indicating:

Missing product descriptions were replaced with “Unknown Product,” preserving transactional integrity for product-level analyses.

Missing CustomerID values were replaced with 0, representing "Guest Customers" in sales analysis, ensuring no loss of sales data due to unidentified customers.

Quantity & UnitPrice :-

The minimum Quantity is 1, confirming removal of zero or negative quantities that indicated returns or cancellations.

The minimum UnitPrice is 0.001, showing that invalid or negative prices have been excluded.

The maximum Quantity (80,995) and UnitPrice (13,541.33) indicate some extremely large transactions or data entry anomalies that may warrant future outlier analysis or capping depending on business rules.

The mean Quantity (10.62) and UnitPrice (3.92) suggest typical average sales quantities and pricing, useful for inventory and revenue forecasting.

CustomerID :-

CustomerID now ranges from 0 (Guest Customer) to 18,287, covering the size of the customer base.

Filling missing IDs as 0 allows segmentation between registered loyal customers and guest purchasers, enabling diverse customer behavior analytics.

Business Impact :-

Removing canceled and invalid transactions ensures that downstream sales trends and inventory insights reflect true revenue-generating activity.

Filling missing data and managing duplicates safeguards data completeness, preventing bias in sales and customer segmentation analyses.

By marking unknown customers distinctly, the analysis can separately address sales driven by registered vs. anonymous buyers, supporting targeted marketing strategies.

# let’s fast-forward and think about Step 3: Load (after cleaning). This is the point where we store the cleaned dataset into a database and design it in a way that supports efficient reporting and dashboarding.

In [18]:
import sqlite3

In [19]:
conn = sqlite3.connect("retail_sales.db")
cursor = conn.cursor()

In [21]:
# Assume cleaned DataFrame is in df (replace df_clean with df)
df.to_sql("FactSales", conn, if_exists="replace", index=False)

524878

In [22]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimCustomer AS
SELECT DISTINCT CustomerID, Country
FROM FactSales
WHERE CustomerID IS NOT NULL
""")


In [23]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimProduct AS
SELECT DISTINCT StockCode, Description, UnitPrice
FROM FactSales
""")

In [24]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimDate AS
SELECT DISTINCT 
    DATE(InvoiceDate) as Date,
    CAST(STRFTIME('%Y', InvoiceDate) AS INTEGER) as Year,
    CAST(STRFTIME('%m', InvoiceDate) AS INTEGER) as Month,
    CAST(STRFTIME('%d', InvoiceDate) AS INTEGER) as Day,
    CASE STRFTIME('%w', InvoiceDate)
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        WHEN '6' THEN 'Saturday'
    END as Weekday
FROM FactSales
""")

In [25]:
conn.commit()

In [26]:
conn.close()

In [27]:
conn = sqlite3.connect("retail_sales.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")  # Enable foreign key enforcement


In [28]:
conn = sqlite3.connect("retail_sales.db")

# Display first 5 rows from FactSales table
df_fact = pd.read_sql_query("SELECT * FROM FactSales LIMIT 5;", conn)
print(df_fact)

# Display first 5 rows from DimCustomer table
df_cust = pd.read_sql_query("SELECT * FROM DimCustomer LIMIT 5;", conn)
print(df_cust)

conn.close()

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55       17850  United Kingdom  
1  2010-12-01 08:26:00       3.39       17850  United Kingdom  
2  2010-12-01 08:26:00       2.75       17850  United Kingdom  
3  2010-12-01 08:26:00       3.39       17850  United Kingdom  
4  2010-12-01 08:26:00       3.39       17850  United Kingdom  
   CustomerID         Country
0       17850  United Kingdom
1       13047  United Kingdom
2       12583          France
3       13748  United Kingdom
4       15100  United Kingdom


Loaded the cleaned retail sales data into an SQLite database as the main fact table FactSales.

The FactSales table holds detailed transaction records: invoice, product, quantity, price, date, customer, and country.

Created dimension tables (DimCustomer, DimProduct, DimDate) using distinct values from the fact table for efficient data organization.

DimCustomer stores unique customers and their countries to enable geographic segmentation.

DimProduct and DimDate enable product-level and time-based analysis, essential for business insights.

The star schema design supports powerful, multi-dimensional queries and aligns with BI best practices.

This structured database model lays the groundwork for scalable analytics and dashboard integration.

# Step 4: ETL Pipeline Automation

Imports and logging setup:



In [29]:
import logging

logging.basicConfig(
    level=logging.INFO,
    filename='etl_log.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s'
)

Define ETL functions (extract, transform, load):

In [30]:
def extract(filepath):
    logging.info("Starting data extraction")
    try:
        df = pd.read_excel(filepath)
        logging.info(f"Extracted {df.shape[0]} rows from {filepath}")
        return df
    except Exception as e:
        logging.error(f"Error during data extraction: {e}")
        raise

def transform(df):
    logging.info("Starting data transformation")
    try:
        df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
        df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
        df['Description'] = df['Description'].fillna('Unknown Product')
        df['CustomerID'] = df['CustomerID'].fillna(0).astype(int)
        before = df.shape[0]
        df = df.drop_duplicates()
        removed = before - df.shape[0]
        logging.info(f"Removed {removed} duplicate rows during transformation")
        logging.info(f"Data shape after transformation: {df.shape}")
        return df
    except Exception as e:
        logging.error(f"Error during data transformation: {e}")
        raise

def load(df, db_path):
    logging.info("Starting data load to database")
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("PRAGMA foreign_keys = ON;")
        df.to_sql("FactSales", conn, if_exists="replace", index=False)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS DimCustomer AS
            SELECT DISTINCT CustomerID, Country
            FROM FactSales
            WHERE CustomerID IS NOT NULL
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS DimProduct AS
            SELECT DISTINCT StockCode, Description, UnitPrice
            FROM FactSales
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS DimDate AS
            SELECT DISTINCT 
                DATE(InvoiceDate) as Date,
                CAST(STRFTIME('%Y', InvoiceDate) AS INTEGER) as Year,
                CAST(STRFTIME('%m', InvoiceDate) AS INTEGER) as Month,
                CAST(STRFTIME('%d', InvoiceDate) AS INTEGER) as Day,
                CASE STRFTIME('%w', InvoiceDate)
                    WHEN '0' THEN 'Sunday'
                    WHEN '1' THEN 'Monday'
                    WHEN '2' THEN 'Tuesday'
                    WHEN '3' THEN 'Wednesday'
                    WHEN '4' THEN 'Thursday'
                    WHEN '5' THEN 'Friday'
                    WHEN '6' THEN 'Saturday'
                END as Weekday
            FROM FactSales
        """)
        conn.commit()
        conn.close()
        logging.info("Data load completed successfully")
    except Exception as e:
        logging.error(f"Error during data load: {e}")
        raise

Define and run the pipeline:

In [34]:
def etl_pipeline():
    raw_file = r"C:\Users\Hp\OneDrive\Desktop\projects\UCI Online Retail Dataset\Online Retail.xlsx"  # full path
    db_file = "retail_sales.db"
    df_raw = extract(raw_file)
    df_clean = transform(df_raw)
    load(df_clean, db_file)


# Run ETL pipeline
etl_pipeline()

# converting it into a csv for powerbi

In [36]:
# Ensure df_clean is defined (replace 'df' with your cleaned DataFrame variable if different)
df_clean = df.copy()

# Export FactSales cleaned data
df_clean.to_csv("FactSales_clean.csv", index=False)

# Create and export DimCustomer
dim_customer = df_clean[['CustomerID', 'Country']].drop_duplicates()
dim_customer.to_csv("DimCustomer.csv", index=False)

# Create and export DimProduct
dim_product = df_clean[['StockCode', 'Description', 'UnitPrice']].drop_duplicates()
dim_product.to_csv("DimProduct.csv", index=False)

# Create and export DimDate
dim_date = df_clean[['InvoiceDate']].drop_duplicates()
dim_date['Year'] = dim_date['InvoiceDate'].dt.year
dim_date['Month'] = dim_date['InvoiceDate'].dt.month
dim_date['Day'] = dim_date['InvoiceDate'].dt.day
dim_date['Weekday'] = dim_date['InvoiceDate'].dt.day_name()
dim_date.to_csv("DimDate.csv", index=False)
